In [22]:
from sklearn import preprocessing
import pandas as pd
import numpy
from numpy import array

In [67]:
from sklearn.ensemble import RandomForestRegressor

In [68]:
# define random forest classifier
forest = RandomForestRegressor(random_state=42, max_depth=5)

In [18]:
def split_dataset(data):
	# split into standard weeks
	train, test = data[1:5011, 1:], data[4000:, 1:]
	# restructure into windows of weekly data
	#train = array(split(train, len(train)/7))
	#test = array(split(test, len(test)/7))
	return train, test

In [76]:
def to_supervised(train, n_input, n_out=5):
	data = train
	X, y = list(), list()
	in_start = 0
	# step over the entire history one time step at a time
	for _ in range(len(data)):
		# define the end of the input sequence
		in_end = in_start + n_input
		out_end = in_end + n_out
		# ensure we have enough data for this instance
		if out_end <= len(data):
			X.append(data[in_start:in_end,:])
			y.append(data[in_end:out_end, 3])
		# move along one time step
		in_start += 1
	return array(X), array(y)

In [96]:
cols = list(pd.read_csv('01.csv',nrows=1, sep=';'))
dataset = pd.read_csv('01.csv', sep=';', engine='python', usecols =[i for i in cols if i != 'Timestamp'])
#split into train and test
#train, test = split_dataset(dataset.values)
#X,y = to_supervised(train,30)
#dataset=numpy.asarray(dataset).astype(numpy.float32)
#y=numpy.asarray(y).astype(numpy.float32)

In [62]:
features = array(['CPU cores',
                     'CPU capacity provisioned [MHZ]', 'CPU usage [MHZ]',
                     'Memory capacity provisioned [KB]', 'Memory usage [KB]','Memory usage [%]', 
                     'Disk read throughput [KB/s]', 'Disk write throughput [KB/s]',
                     'Disk size [GB]', 'Network received throughput [KB/s]', 'Network transmitted throughput [KB/s]'])

In [104]:
forest.fit(numpy.asarray(dataset).astype(numpy.float32),numpy.asarray(dataset).astype(numpy.float32)[:, 6])

RandomForestRegressor(max_depth=5, n_estimators=63,
                      random_state=RandomState(MT19937) at 0x7FA10E8DD340)

In [98]:
from boruta import BorutaPy

In [105]:
boruta = BorutaPy(
   estimator = forest, 
   n_estimators = 'auto',
   max_iter = 100 # number of trials to perform
)

In [106]:
boruta.fit(numpy.asarray(dataset).astype(numpy.float32),numpy.asarray(dataset).astype(numpy.float32)[:, 6])

BorutaPy(estimator=RandomForestRegressor(max_depth=5, n_estimators=48,
                                         random_state=RandomState(MT19937) at 0x7FA10E8DD340),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x7FA10E8DD340)

In [107]:
green_area = dataset.columns[boruta.support_].to_list()

In [108]:
blue_area = dataset.columns[boruta.support_weak_].to_list()

In [109]:
print('features in the green area:', green_area)
print('features in the blue area:', blue_area)

features in the green area: ['Memory usage [KB]', 'Memory usage [%]']
features in the blue area: []
